In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from requests import get
import time
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

In [25]:
def get_urls(start_page,stop_page):
    pages = list(range(start_page,stop_page+1))
    page_urls = []
    for page in pages:
        p = f'https://github.com/search?o=desc&p={page}&q=stars%3A%3E1&s=stars&type=Repositories'
        page_urls.append(p)
    repo_links_all = []
    time.sleep(3)
    for page in page_urls:
        url = page
        headers  = {'User-Agent' : 'Codeup Data Science Student'} #user-agent to decrease chance of being rejected by github
        response = get(url,headers)
        soup = BeautifulSoup(response.text) #created a BS object
        repo_list = soup.find('ul', class_='repo-list') #find section of text of interest
        links = repo_list.find_all(href = True) #isolate the 'href's 
        repo_links= [link['href'] for link in links if not re.match(r'^/topics',link['href']) and not re.match(r'^https',link['href'])\
                 and link['href'].count('/') < 3] #further pare down list by excluding anything that begins with /topics, https or more than 2 forward slashes
        repo_links_all.extend(repo_links)
    repo_links_all = [url[1:] for url in repo_links_all]
    return repo_links_all

In [36]:
urls = get_urls(1,7)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [37]:
urls2 = get_urls(8,14)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [31]:
urls3 = get_urls(15,22)

In [35]:
urls3

80

In [32]:
repo_urls = urls + urls2 + urls3

NameError: name 'urls' is not defined

In [21]:
from env import github_token
import os
import json
from typing import Dict, List
import requests

repos = url_list

headers = {
    "Authorization": f"token {github_token}",
    "User-Agent": "soslin",
}

if (
    headers["Authorization"] == "token "
    or headers["User-Agent"] == "YOUR_GITHUB_USERNAME"
):
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )
def github_api_request(url: str) -> requests.Response:
    return requests.get(url, headers=headers)


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    return github_api_request(url).json()["language"]


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    return github_api_request(url).json()


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists
    the files in a repo and returns the url that can be
    used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns
    a dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": requests.get(get_readme_download_url(contents)).text,
    }


def scrape_github_data():
    """
    Loop through all of the repos and process them. Saves the data in
    `data.json`.
    """
    data = [process_repo(repo) for repo in repos]
    json.dump(data, open("data.json", "w"))



scrape = scrape_github_data()

In [22]:
scrape

In [40]:
df = pd.read_json('data.json')

In [41]:
df

""


In [42]:
def basic_clean(string):
    """
    Lowercase the string
    Normalize unicode characters
    Replace anything that is not a letter, number, whitespace or a single quote.
    """
    string = string.lower()
    string = unicodedata.normalize('NFKD', string).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # remove anything not a space character, an apostrophy, letter, or number
    string = re.sub(r"[^a-z0-9'\s]", '', string)

    # convert newlines and tabs to a single space
    string = re.sub(r'[\r|\n|\r\n]+', ' ', string)
    
    string = string.strip()
    return string

In [43]:
df2 = df.copy()
df2.head()

""


In [39]:
df2.set_index('repo', inplace = True) 

KeyError: 'repo'

In [13]:
df2.head()

,language,readme_contents
repo,,
gocodeup/codeup-setup-script,Shell,# Codeup Setup Script\n\nSetup script for Code...
gocodeup/movies-application,JavaScript,"# Movies Application\n\nFor this project, we w..."


In [14]:
def tokenize(s):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    return tokenizer.tokenize(s, return_str=True)

In [15]:
tokenize(df2)

'language \\\nrepo \ngocodeup/codeup-setup-script Shell \ngocodeup/movies-application JavaScript \n\n readme_contents \nrepo \ngocodeup/codeup-setup-script # Codeup Setup Script\\n\\nSetup script for Code ... \ngocodeup/movies-application # Movies Application\\n\\nFor this project , we w ...'

In [16]:
def lemmatize(s):
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in s.split()]
    string_of_lemmas = ' '.join(lemmas)
    return string_of_lemmas

In [17]:
def remove_stopwords(s,extra_words =[], exclude_words = []):
    #Tokenize the string
    s = tokenize(s)
    
    words = s.split()
    stopword_list = stopwords.words('english')
    
    #remove the excluded words fro the stopword list
    stopword_list = set(stopword_list) - set(exclude_words)
    
    #add in the user specified extra words
    stopword_list = stopword_list.union(set(extra_words))
    
    filtered_words = [w for w in words if w not in stopword_list]
    final_string = ' '.join(filtered_words)
    return final_string

In [18]:
stopwords_list = []
words_to_exclude = []

In [19]:
def prep_urls(df):
    df['original'] = df.body
    df['stemmed'] = df.body.apply(basic_clean).apply(stem)
    df['lemmatized'] = df.body.apply(basic_clean).apply(lemmatize)
    df['clean'] = df.body.apply(basic_clean).apply(remove_stopwords)
    df.drop(columns = ['body'], inplace = True)
    return df